In [39]:
import pandas as pd
import os

# original file:

###  Missing dates and times in 2017:

                 
# Normalized file

###  Missing dates and times in 2017:
       
           
##### both of them miss:  03-08-2020 at 2:00:00

In [40]:
#orignal file
from datetime import date
def add_rows1(file):
    df = pd.read_excel(file)
    df['Hour'] = df['Time'].map(lambda x: x.hour)
    Zone = list(df['Zone'].unique())[0] #Zone name
    
    # hourly average:
    df = df.groupby(['Date', 'Hour']).mean() #Datetime,Time, Zone columns will be deleted by this
    df = df.reset_index()
    
# If there is no missing date, then don't use the code below
    # problems only occur these in July and Oct in 2017:
    df_July = df[(df['Year']== 2017) &(df['Month']== 7)]
    # df_Oct = df[(df['Year']== 2017) &(df['Month']== 10)]
    
    #Lets's find which day and what time:
    Day_list = list(df['Day'].unique())
    Hour_list = list(df['Hour'].unique())
    
    #July:
    for i in Day_list:
        for j in Hour_list:
            df_cut = df_July[(df_July['Day']== i) & (df_July['Hour']== j)] 
            if len(df_cut) == 0:
                df.loc[df.shape[0]] = dict(zip(df.columns, 
                                                 (date(2017,7,i), j, 2017, 7, i,
                                                  None, None, None, None, None)))
                
                
    #Oct:
#     for i in Day_list:
#         for j in Hour_list:
#             df_cut = df_Oct[(df_Oct['Day']== i) & (df_Oct['Hour']== j)] 
#             if len(df_cut) == 0:
#                 df.loc[df.shape[0]] = dict(zip(df.columns, 
#                                                  (date(2017,10,i), j, 2017, 10, i,
#                                                   None, None, None, None, None)))
                
    #2017 Zone S:
    solar2017 = df.loc[(df['Date'] < date(2017,7,1))]
    if (len(solar2017) == 0) & (Zone == 'S'):
        df_add_raw = pd.DataFrame(columns = solar2017.columns) # create a dateframe
        idx = pd.date_range(start = '2017-01-01', end = '2017-6-30 23:00:00', freq = 'H')
        df_add_raw['Date'] = idx.date
        df_add_raw['Year'] = idx.year
        df_add_raw['Month'] = idx.month
        df_add_raw['Day'] = idx.day
        df_add_raw['Hour'] = idx.hour
        df = df.append(df_add_raw, ignore_index = True)
      
    df = df.sort_values(by = ['Date','Hour']).reset_index(drop = True)
    
    return df, Zone

In [41]:
# normalized file
def add_rows2(file):
    df = pd.read_excel(file)
    
    df['HourlyInt'] = pd.to_datetime(df['HourlyInt'])
    df['Month'] =  df['HourlyInt'].dt.month
    df['Day'] =  df['HourlyInt'].dt.day
    df['Hour'] =  df['HourlyInt'].dt.hour
    
# If there is no missing date, then don't use the code below
    # problems only occur in Oct 2017:
    df_Oct = df[(df['Year']== 2017)&(df['Month']== 10)]
    
    #Lets's find which day and what time:
    Day_list = list(df['Day'].unique())
    Hour_list = list(df['Hour'].unique())                
                
#     #Oct:
#     for i in Day_list:
#         for j in Hour_list:
#             df_cut = df_Oct[(df_Oct['Day']== i) & (df_Oct['Hour']== j)] 
#             if len(df_cut) == 0:
#                 df.loc[df.shape[0]] = dict(zip(df.columns, 
#                                                (date(2017,10,i), 2017, None, None, None, None, None, None,
#                                                None, None, None,None, None,None, 10,i, j)))
            
                
    df = df.sort_values(by = ['Year','Month', 'Day', 'Hour']).reset_index(drop = True) 
    
    return df

# Swap

In [73]:
def swap(file1, file2):
    df1, Zone = add_rows1(file1)

    df2 = add_rows2(file2)
    df2 = df2[['%s'%(Zone), 'Year', 'Month', 'Day', 'Hour']]
    df1['BTM'] = df2['%s'%(Zone)] #swap
    df_swap = pd.merge(df1, df2, on = ['Year', 'Month', 'Day', 'Hour']) #merge

    del df_swap['BTM']
    df_swap.rename(columns = {'%s'%(Zone): 'BTM'}, inplace = True)
    
    # add a row:
    df_swap.loc[df_swap.shape[0]] = dict(zip(df_swap.columns, 
                                   (date(2020,3,8), 2, 2020, 3, 8, None, None,None,None,None)))
    #sort
    df_swap = df_swap.sort_values(by = ['Year','Month', 'Day', 'Hour']).reset_index(drop = True)
    #change format
    df_swap['Date'] = pd.to_datetime(df_swap['Date']) 

    return df_swap

In [62]:
df1 = swap('C:/Users/zhongj/Desktop/Data/Zonal_SolarData/ZoneS_SolarData.xlsx', 'C:/Users/zhongj/Desktop/Data/Normalized_Solar.xlsx')

# Merge

In [63]:
def Merge_file(df, file):
    df2 = pd.read_excel(file)
    df2['Hour'] = df2['Time'].map(lambda x: x.hour) #get hour
    
    df_merge = pd.merge(df2, df, on = ['Date','Hour']) #merge
    
    df['HrFlag'] = df['Hour'].map(lambda x: 1 if x >= 6 and x <=19  else 0)
    
    # change columns order
    df_merge = df_merge[['Zone', 'Date','Year', 'Month', 'Day','Hour','Time','HrFlag',
           'BTM', 'GHI', 'POA20', 'POA30',
           '_CLC', '_DBT60', '_DEW', '_GHI60', '_SSM','_THI', '_WCI', '_WDR', '_WET', '_WSP' ]]
    
    del df_merge['Date']
    del df_merge['Time']
    
    return df_merge
   

In [64]:
df = Merge_file(df1, 'C:/Users/zhongj/Desktop/Data/Zonal_WeatherData/ZoneS_WeatherData.xlsx')

# All files

In [74]:
files_dir_solar = 'C:/Users/zhongj/Desktop/Data/Zonal_SolarData' 

files_dir_weather = "C:/Users/zhongj/Desktop/Data/Zonal_WeatherData" 

normalized_solar =  'C:/Users/zhongj/Desktop/Data/Normalized_Solar.xlsx'

final_files_dir = "C:/Users/zhongj/Desktop/Final_Zonal_Data" 

for filename1 in os.listdir(files_dir_solar):
    for filename2 in os.listdir(files_dir_weather):
        if filename1.split("_")[0] == filename2.split("_")[0]:
            file_path1 = os.path.join(files_dir_solar, filename1)
            file_path2 = os.path.join(files_dir_weather, filename2)
            df1 = swap(file_path1 , normalized_solar)
            df_final = Merge_file(df1, file_path2)
            
            #output:
            os.chdir(final_files_dir)
            
            df_final.to_excel(r"%s_FinalData.xlsx"%(filename1.split("_")[0]), 'sheet1',index=False) # create a excel file
